# Hello Tensorflow Too

## test 0 alt: Fashion MNIST classification with tf.data.Datasets & tf.keras

here we create a small 2D convolutional network to classify Fashion MNIST clothing dataset.

it consists of a number of 2D convolutional layers that compress the 28x28 pixel digits to 64-dim vector before a final dense network for classification.

we can then test using this classification network as the encoder of an autoencoder model (next notebook).


In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0-beta1'

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# add axis for channel and normalize to 0 ~ 1 (based on training max)
x_test = np.expand_dims(x_test, axis=-1)/np.max(x_train)
x_train = np.expand_dims(x_train, axis=-1)/np.max(x_train)
x_shape = x_train[0].shape
print(np.min(x_train), np.max(x_train))
print(x_train.shape)
# add axis for sparse categorical crossentropy (unnecessary)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)
print(y_train.shape)

0.0 1.0
(60000, 28, 28, 1)
(60000, 1)


In [4]:
trn_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
trn_dataset = trn_dataset.shuffle(buffer_size=128).batch(32, drop_remainder=True)

In [5]:
tst_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
tst_dataset = tst_dataset.shuffle(buffer_size=128).batch(32, drop_remainder=True)

In [6]:
clfmdl = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, kernel_size=2, activation='relu', strides=2, input_shape=x_shape, name='conv2d_a'),
    tf.keras.layers.Dropout(rate=0.2, name='dropout_a'),
    tf.keras.layers.Conv2D(8, kernel_size=4, activation='relu', strides=1, name='conv2d_b'),
    tf.keras.layers.Conv2D(8, kernel_size=4, activation='relu', strides=1, name='conv2d_c'),
    tf.keras.layers.Dropout(rate=0.2, name='dropout_b'),
    tf.keras.layers.Conv2D(8, kernel_size=4, activation='relu', strides=1, name='conv2d_d'),
    tf.keras.layers.Conv2D(4, kernel_size=2, activation='tanh', name='conv2d_e'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(10, activation='softmax', name='dense')
], name='classifier')

In [7]:
clfmdl.summary()

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_a (Conv2D)            (None, 14, 14, 8)         40        
_________________________________________________________________
dropout_a (Dropout)          (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_b (Conv2D)            (None, 11, 11, 8)         1032      
_________________________________________________________________
conv2d_c (Conv2D)            (None, 8, 8, 8)           1032      
_________________________________________________________________
dropout_b (Dropout)          (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_d (Conv2D)            (None, 5, 5, 8)           1032      
_________________________________________________________________
conv2d_e (Conv2D)            (None, 4, 4, 4)           1

In [8]:
clfmdl.compile(optimizer=tf.keras.optimizers.Adam(),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [9]:
clfmdl.fit(trn_dataset, epochs=10)

W0621 10:36:16.213540 140273939883776 deprecation.py:323] From /home/derek/miniconda3/envs/tf2beta/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.5049 - sparse_categorical_accuracy: 0.8425
Epoch 2/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1845 - sparse_categorical_accuracy: 0.9433
Epoch 3/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1342 - sparse_categorical_accuracy: 0.9584
Epoch 4/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1136 - sparse_categorical_accuracy: 0.9648
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1003 - sparse_categorical_accuracy: 0.9689
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0935 - sparse_categorical_accuracy: 0.9707
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0861 - sparse_categorical_accuracy: 0.9726
Epoch 8/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0835 - sparse_categorical_accuracy: 0.9737
Epoch 9/10
1875/1875 [==

In [10]:
clfmdl.evaluate(tst_dataset)

312/312 [==============================] - 1s 3ms/step - loss: 0.0592 - sparse_categorical_accuracy: 0.9805


[0.05921341217822956, 0.98046875]

## results

after ten epochs, this network reaches 98% accuracy on the test set